# **Previsión de Series Temporales con RNN (LSTM y GRU) en Keras**
## **Dataset: Pasajeros de Aerolínea (Air Passengers)**

**Objetivo del Notebook:**
Este notebook presenta un flujo de trabajo para construir, entrenar, evaluar y utilizar modelos de Redes Neuronales Recurrentes (RNNs), específicamente LSTM y GRU, para la previsión de series temporales utilizando **Keras (con TensorFlow backend)**.

**Estructura del Notebook:**
1.  Carga de las librerías necesarias.
2.  Carga del dataset de series temporales (Air Passengers desde CSV).
3.  Visualización del dataset y Análisis Exploratorio de Datos (EDA).
4.  Preprocesamiento del dataset (escalado, creación de secuencias).
5.  Definición y configuración de dos modelos RNN (LSTM y GRU) con Keras.
6.  Entrenamiento y evaluación de los modelos.
7.  Inferencia (previsión) con el mejor modelo.
8.  Guardado del mejor modelo.

In [ ]:
# 1. Carga de las Librerías
# ------------------------------------------------------------------------------

# TensorFlow y Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam

# Pandas para la carga y manipulación de datos CSV
import pandas as pd

# NumPy para operaciones numéricas
import numpy as np

# Scikit-learn para preprocesamiento (escalado) y métricas
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Matplotlib para visualización
import matplotlib.pyplot as plt
import matplotlib.dates as mdates # Para formatear fechas en los gráficos

# Otras utilidades
import os
import datetime
import math # Para sqrt en RMSE

# Comprobación de versiones
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")

# Configuraciones opcionales para Matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6] # Ajustar tamaño para series temporales
plt.rcParams['font.size'] = 12

# Determinar el dispositivo (GPU si está disponible, sino CPU)
# Keras/TensorFlow manejan esto más automáticamente.
# Podemos verificar la disponibilidad de GPU si es necesario.
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\nUsando dispositivo: GPU ({len(gpus)} GPUs disponibles)")
    # tf.config.experimental.set_memory_growth(gpus[0], True) # Opcional: para evitar que TF reserve toda la memoria GPU
else:
    print("\nUsando dispositivo: CPU")

## **2. Carga del Dataset de Series Temporales**

En esta sección, cargaremos el dataset "Air Passengers".
Asumiremos que está en un archivo CSV llamado `AirPassengers.csv` con columnas "Month" y "#Passengers".
Realizaremos una inspección inicial de los datos.

**Nota:** Si no tienes el archivo `AirPassengers.csv`, puedes buscarlo online (es un dataset muy común)
o usar otros datasets de series temporales disponibles.

In [ ]:
# 2. Carga del Dataset
# ------------------------------------------------------------------------------

# --- Constantes Globales Clave ---
# Esta constante BATCH_SIZE es fundamental y se utiliza en celdas posteriores
# (en model.fit())
BATCH_SIZE = 32
# --- Fin de Constantes Globales Clave ---

# Ruta al archivo CSV. Ajusta esta ruta si es necesario.
# Comúnmente, este archivo tiene columnas 'Month' y '#Passengers' o similar.
csv_path = 'AirPassengers.csv' # ASEGÚRATE DE QUE ESTE ARCHIVO EXISTA EN LA RUTA ESPECIFICADA

try:
    # Cargar el dataset
    df = pd.read_csv(csv_path)
    print(f"Dataset '{csv_path}' cargado exitosamente.")

    # Inspección inicial
    print("\n--- Primeras filas del dataset: ---")
    print(df.head())

    print("\n--- Información del DataFrame: ---")
    df.info()

    # Asumir que la columna de pasajeros es la segunda (índice 1) si no se llama '#Passengers'
    # y la columna de fecha es la primera (índice 0)
    if '#Passengers' in df.columns:
        passengers_col_name = '#Passengers'
    elif 'Passengers' in df.columns:
        passengers_col_name = 'Passengers'
    else:
        print(f"Advertencia: No se encontró la columna '#Passengers' o 'Passengers'. Usando la segunda columna: {df.columns[1]}")
        passengers_col_name = df.columns[1]

    if 'Month' in df.columns:
        date_col_name = 'Month'
    else:
        print(f"Advertencia: No se encontró la columna 'Month'. Usando la primera columna: {df.columns[0]}")
        date_col_name = df.columns[0]

    # Convertir la columna de fecha a datetime y establecerla como índice
    df[date_col_name] = pd.to_datetime(df[date_col_name])
    df.set_index(date_col_name, inplace=True)

    # Seleccionar solo la serie de pasajeros
    time_series_data = df[passengers_col_name]

    print(f"\n--- Serie temporal de '{passengers_col_name}' (primeras filas): ---")
    print(time_series_data.head())

except FileNotFoundError:
    print(f"Error: El archivo '{csv_path}' no fue encontrado.")
    print("Por favor, descarga el dataset 'AirPassengers.csv' y colócalo en la ruta correcta o actualiza 'csv_path'.")
    # Crear un DataFrame de ejemplo si el archivo no se encuentra, para que el resto del notebook pueda ejecutarse parcialmente.
    print("Creando un dataset de ejemplo para demostración...")
    dates_example = pd.date_range(start='1949-01-01', periods=144, freq='MS') # 'MS' para inicio de mes
    values_example = np.linspace(100, 600, 144) + np.random.randn(144)*20 + np.sin(np.arange(144)/6)*50
    values_example = np.maximum(50, values_example).astype(int) # asegurar positivos
    df = pd.DataFrame({'Month': dates_example, '#Passengers': values_example})
    df['Month'] = pd.to_datetime(df['Month'])
    df.set_index('Month', inplace=True)
    passengers_col_name = '#Passengers'
    time_series_data = df[passengers_col_name]
    print("Dataset de ejemplo creado.")

## **3. Visualización del Dataset y Análisis Exploratorio de Datos (EDA)**

Visualizaremos la serie temporal para identificar patrones como tendencia, estacionalidad y posibles anomalías.
También verificaremos si hay valores faltantes.

In [ ]:
# 3. Visualización del Dataset y EDA
# ------------------------------------------------------------------------------

if 'time_series_data' in locals():
    # 3.1 Visualización de la Serie Temporal Completa
    print("\n--- 3.1 Visualización de la Serie Temporal Completa ---")
    plt.figure(figsize=(14, 7))
    plt.plot(time_series_data.index, time_series_data.values, label=f'Número de {passengers_col_name}')
    plt.title(f'Serie Temporal: {passengers_col_name} (1949-1960)', fontsize=16)
    plt.xlabel('Fecha', fontsize=12)
    plt.ylabel(f'Número de {passengers_col_name}', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    # Formatear el eje x para mostrar años
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.gca().xaxis.set_major_locator(mdates.YearLocator(2)) # Marcas cada 2 años
    plt.show()

    # 3.2 Verificación de Valores Faltantes
    print("\n--- 3.2 Verificación de Valores Faltantes ---")
    missing_values = time_series_data.isnull().sum()
    print(f"Número de valores faltantes en la serie: {missing_values}")
    if missing_values > 0:
        print("Advertencia: Existen valores faltantes. Se deberían tratar (ej. interpolación, eliminación).")
        # Para este ejemplo, no implementaremos el tratamiento de faltantes.
else:
    print("No se pudo cargar `time_series_data`. Saltando EDA.")

## **4. Preprocesamiento del Dataset**

El preprocesamiento para modelos RNN de series temporales generalmente incluye:
1.  **Escalado de Datos:** Los modelos de redes neuronales suelen funcionar mejor con datos escalados (ej. entre 0 y 1 o -1 y 1). Usaremos `MinMaxScaler`.
2.  **División en Entrenamiento y Prueba:** Es crucial que la división sea cronológica para series temporales. No se deben mezclar datos del futuro en el conjunto de entrenamiento.
3.  **Creación de Secuencias:** Las RNNs (LSTM/GRU) procesan secuencias de datos. Necesitamos transformar nuestra serie temporal en pares de (secuencia_de_entrada, valor_objetivo).
    * `sequence_length` (o lookback): Número de pasos temporales pasados que se usarán como entrada.
    * `target_steps` (o horizon): Número de pasos futuros a predecir (aquí, 1).

In [ ]:
# 4. Preprocesamiento del Dataset
# ------------------------------------------------------------------------------

if 'time_series_data' in locals():
    # 4.1 Escalado de Datos
    print("\n--- 4.1 Escalado de Datos ---")
    scaler = MinMaxScaler(feature_range=(-1, 1)) # Escalar a [-1, 1] es común para LSTMs
    # scaler = MinMaxScaler(feature_range=(0, 1)) # Escalar a [0, 1] también es una opción

    # El método fit_transform espera un array 2D, así que convertimos la serie 1D
    data_scaled = scaler.fit_transform(time_series_data.values.reshape(-1, 1))
    print(f"Forma de los datos escalados: {data_scaled.shape}")
    print(f"Primeros 5 valores escalados:\n{data_scaled[:5]}")

    # 4.2 División en Entrenamiento y Prueba (Cronológica)
    print("\n--- 4.2 División en Entrenamiento y Prueba ---")
    # Por ejemplo, usar el 80% para entrenamiento y el 20% para prueba
    train_size = int(len(data_scaled) * 0.80)
    test_size = len(data_scaled) - train_size

    train_data_scaled = data_scaled[0:train_size, :]
    test_data_scaled = data_scaled[train_size:len(data_scaled), :]

    print(f"Tamaño del conjunto de entrenamiento escalado: {len(train_data_scaled)}")
    print(f"Tamaño del conjunto de prueba escalado: {len(test_data_scaled)}")

    # 4.3 Creación de Secuencias
    print("\n--- 4.3 Creación de Secuencias ---")
    def create_sequences(input_data, sequence_length, target_steps=1):
        """
        Crea secuencias de entrada y etiquetas objetivo para modelos RNN.
        Args:
            input_data (np.array): Array de la serie temporal (ya escalada).
            sequence_length (int): Número de pasos temporales pasados a usar como entrada.
            target_steps (int): Número de pasos futuros a predecir. Default 1.
        Returns:
            tuple: (np.array de secuencias de entrada, np.array de etiquetas objetivo)
        """
        X, y = [], []
        for i in range(len(input_data) - sequence_length - (target_steps - 1)):
            # La secuencia de entrada
            seq_in = input_data[i:(i + sequence_length), 0] # Tomar 'sequence_length' valores
            # La etiqueta objetivo (el/los siguiente/s valor/es)
            seq_out = input_data[(i + sequence_length):(i + sequence_length + target_steps), 0]
            X.append(seq_in)
            y.append(seq_out)
        return np.array(X), np.array(y)

    SEQUENCE_LENGTH = 12  # Usar los últimos 12 meses para predecir el siguiente
    TARGET_STEPS = 1      # Predecir 1 mes adelante
    NUM_FEATURES = 1      # Serie temporal univariada

    X_train, y_train = create_sequences(train_data_scaled, SEQUENCE_LENGTH, TARGET_STEPS)
    X_test, y_test = create_sequences(test_data_scaled, SEQUENCE_LENGTH, TARGET_STEPS)

    print(f"Forma de X_train: {X_train.shape}") # Debería ser (num_samples, sequence_length)
    print(f"Forma de y_train: {y_train.shape}")   # Debería ser (num_samples, target_steps)
    print(f"Forma de X_test: {X_test.shape}")
    print(f"Forma de y_test: {y_test.shape}")

    # Ajustar la forma de X para Keras RNNs: (num_samples, sequence_length, num_features)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], NUM_FEATURES)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], NUM_FEATURES)
    # y_train e y_test ya tienen la forma correcta (num_samples, target_steps) para Dense(target_steps)

    print(f"\nForma de X_train para Keras: {X_train.shape}")
    print(f"Forma de y_train para Keras: {y_train.shape}")

else:
    print("No se pudo cargar `time_series_data`. Saltando Preprocesamiento.")
    # Definir arrays vacíos para que el resto del notebook no falle catastróficamente
    X_train, y_train, X_test, y_test = np.array([]), np.array([]), np.array([]), np.array([])


print("\n--- Fin de Preprocesamiento ---")

## **5. Definición de Modelos RNN (LSTM y GRU) con Keras**

Definiremos dos modelos basados en RNN usando Keras `Sequential` API:
1.  **Modelo LSTM:** Utilizará una o más capas LSTM seguidas de una capa Densa para la predicción.
2.  **Modelo GRU:** Similar, pero utilizando capas GRU.

Ambos modelos tomarán secuencias de entrada y predecirán el valor del siguiente paso temporal.
Las capas LSTM y GRU en Keras esperan la entrada como `(batch_size, timesteps, input_features)`.
Nuestra `input_features` (número de features por paso temporal) es 1 para esta serie univariada.

In [ ]:
# 5. Definición de Modelos RNN (LSTM y GRU) con Keras
# ------------------------------------------------------------------------------

INPUT_SIZE_RNN = NUM_FEATURES # Número de features en cada paso temporal (1 para univariada)
HIDDEN_SIZE_RNN = 64 # Número de neuronas en la capa oculta de LSTM/GRU
# NUM_LAYERS_RNN = 1   # Keras LSTM/GRU lo maneja apilando capas si es necesario
OUTPUT_SIZE_RNN = TARGET_STEPS # Predecir `TARGET_STEPS` adelante (aquí 1)
# SEQUENCE_LENGTH ya está definido

def crear_modelo_lstm_keras(input_shape, hidden_units, output_units, dropout_rate=0.0):
    model = Sequential()
    model.add(LSTM(hidden_units, input_shape=input_shape, dropout=dropout_rate, recurrent_dropout=dropout_rate)) # recurrent_dropout solo si GPU lo soporta eficientemente
    # Si NUM_LAYERS_RNN > 1, se añadirían más capas LSTM con return_sequences=True (excepto la última LSTM)
    # model.add(LSTM(hidden_units, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    # model.add(LSTM(hidden_units, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    model.add(Dense(output_units))
    return model

def crear_modelo_gru_keras(input_shape, hidden_units, output_units, dropout_rate=0.0):
    model = Sequential()
    model.add(GRU(hidden_units, input_shape=input_shape, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    model.add(Dense(output_units))
    return model

# Definir input_shape para la primera capa
input_shape_rnn = (SEQUENCE_LENGTH, INPUT_SIZE_RNN)

# Instanciar los modelos
# El dropout en Keras LSTM/GRU se aplica a las conexiones recurrentes si recurrent_dropout > 0
# y a las conexiones de entrada si dropout > 0.
# Para una sola capa, el dropout en PyTorch nn.LSTM/GRU solo se aplicaba si num_layers > 1
# Mantendremos dropout=0 para una sola capa para emular el comportamiento de PyTorch
# Si se usaran múltiples capas apiladas (NUM_LAYERS_RNN > 1), se podría añadir dropout.
dropout_val = 0.2 if False else 0.0 # Cambiar a True para activar dropout si se apilan capas

modelo_lstm_keras = crear_modelo_lstm_keras(input_shape_rnn, HIDDEN_SIZE_RNN, OUTPUT_SIZE_RNN, dropout_rate=dropout_val)
modelo_gru_keras = crear_modelo_gru_keras(input_shape_rnn, HIDDEN_SIZE_RNN, OUTPUT_SIZE_RNN, dropout_rate=dropout_val)

print("--- Modelo LSTM (Keras) ---")
modelo_lstm_keras.summary()

print("\n--- Modelo GRU (Keras) ---")
modelo_gru_keras.summary()

print("\nModelos RNN (Keras) definidos.")

## **6. Entrenamiento y Evaluación de los Modelos (Keras)**

El proceso con Keras es más directo:
1.  Compilar el modelo: Especificar optimizador, función de pérdida y métricas.
    * Función de pérdida: `mean_squared_error` (MSE).
    * Optimizador: `Adam`.
    * Métricas: `mse`.
2.  Entrenar el modelo usando `model.fit()`, que devuelve un objeto `history`.
3.  Evaluar el modelo usando `model.evaluate()`.
4.  Graficar las curvas de pérdida de entrenamiento y validación del objeto `history`.

Las métricas principales serán MSE y RMSE (Raíz del Error Cuadrático Medio).

In [ ]:
# 6. Entrenamiento y Evaluación de los Modelos RNN (Keras)
# ------------------------------------------------------------------------------

criterion_keras = 'mean_squared_error'

def entrenar_y_evaluar_rnn_keras(modelo_rnn, train_X_data, train_y_data, test_X_data, test_y_data,
                                criterio_loss_str, optimizador_rnn, num_epochs_val, batch_size_val,
                                nombre_modelo_str_val):
    print(f"\n--- Entrenando y Evaluando: {nombre_modelo_str_val} ---")

    modelo_rnn.compile(optimizer=optimizador_rnn, loss=criterio_loss_str, metrics=['mse'])

    # El entrenamiento se realiza directamente con los arrays NumPy
    history = modelo_rnn.fit(
        train_X_data, train_y_data,
        epochs=num_epochs_val,
        batch_size=batch_size_val,
        validation_data=(test_X_data, test_y_data),
        verbose=1, # 0 para silencioso, 1 para barra de progreso, 2 para una línea por época
        shuffle=True # shuffle=True es bueno para entrenamiento
    )

    # Graficar historial de pérdida
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
    plt.plot(history.history['val_loss'], label='Pérdida Validación')
    plt.title(f"Historial de Pérdida: {nombre_modelo_str_val}", fontsize=16)
    plt.xlabel('Época', fontsize=12); plt.ylabel('Pérdida (MSE)', fontsize=12)
    plt.legend(); plt.grid(True, linestyle='--', alpha=0.7); plt.show()

    # Encontrar la mejor pérdida de validación y la época
    best_val_loss_keras = min(history.history['val_loss'])
    best_epoch_keras = history.history['val_loss'].index(best_val_loss_keras) + 1
    best_val_rmse_keras = math.sqrt(best_val_loss_keras)

    print(f"\nMejor resultado para {nombre_modelo_str_val} en época {best_epoch_keras}: "
          f"Pérdida Val (MSE) = {best_val_loss_keras:.6f}, RMSE Val = {best_val_rmse_keras:.4f}")

    return modelo_rnn, history, best_val_loss_keras, best_val_rmse_keras


EPOCHS_RNN = 150 # Aumentar para mejores resultados, ej. 100-200
LEARNING_RATE_RNN = 0.001

if X_train.size > 0: # Solo si hay datos
    # Entrenar Modelo LSTM
    optimizer_lstm_keras = Adam(learning_rate=LEARNING_RATE_RNN)
    modelo_lstm_keras, hist_lstm_keras, loss_lstm_keras, rmse_lstm_keras = entrenar_y_evaluar_rnn_keras(
        modelo_lstm_keras, X_train, y_train, X_test, y_test,
        criterion_keras, optimizer_lstm_keras,
        EPOCHS_RNN, BATCH_SIZE, "LSTM (Keras)"
    )

    # Entrenar Modelo GRU
    optimizer_gru_keras = Adam(learning_rate=LEARNING_RATE_RNN)
    modelo_gru_keras, hist_gru_keras, loss_gru_keras, rmse_gru_keras = entrenar_y_evaluar_rnn_keras(
        modelo_gru_keras, X_train, y_train, X_test, y_test,
        criterion_keras, optimizer_gru_keras,
        EPOCHS_RNN, BATCH_SIZE, "GRU (Keras)"
    )

    print("\n--- Resumen de Resultados (Keras - Series Temporales) ---")
    print(f"LSTM: Mejor Pérdida Val (MSE) = {loss_lstm_keras:.6f}, RMSE Val = {rmse_lstm_keras:.4f}")
    print(f"GRU:  Mejor Pérdida Val (MSE) = {loss_gru_keras:.6f}, RMSE Val = {rmse_gru_keras:.4f}")

    # Seleccionar el mejor modelo
    if loss_lstm_keras < loss_gru_keras: # Menor MSE es mejor
        mejor_modelo_rnn_keras = modelo_lstm_keras
        nombre_mejor_modelo_rnn_keras = "LSTM (Keras)"
        rmse_mejor_modelo_rnn_keras = rmse_lstm_keras
    else:
        mejor_modelo_rnn_keras = modelo_gru_keras
        nombre_mejor_modelo_rnn_keras = "GRU (Keras)"
        rmse_mejor_modelo_rnn_keras = rmse_gru_keras
    print(f"\nEl mejor modelo RNN (Keras) es: {nombre_mejor_modelo_rnn_keras} con RMSE Val = {rmse_mejor_modelo_rnn_keras:.4f}")
else:
    print("No se pudieron entrenar los modelos RNN debido a datos de entrenamiento vacíos.")
    mejor_modelo_rnn_keras, nombre_mejor_modelo_rnn_keras = None, "N/A"

## **7. Inferencia (Previsión) con el Mejor Modelo (Keras)**

Usaremos el mejor modelo RNN entrenado para hacer previsiones sobre el conjunto de prueba.

Los pasos son:
1. Usar `model.predict()` sobre los datos de prueba (`X_test`).
2. **Importante:** Desescalar las predicciones y los valores verdaderos para interpretarlos en la escala original de pasajeros.
3. Graficar las predicciones contra los valores reales.

In [ ]:
# 7. Inferencia (Previsión) con el Mejor Modelo (Keras)
# ------------------------------------------------------------------------------

if mejor_modelo_rnn_keras is not None and X_test.size > 0 and 'scaler' in locals():
    print(f"Usando: {nombre_mejor_modelo_rnn_keras} para la previsión.")
    
    # Obtener todas las predicciones del conjunto de prueba
    test_preds_scaled_np = mejor_modelo_rnn_keras.predict(X_test)
    # y_test ya son los labels escalados en formato NumPy

    # Desescalar
    # `scaler` fue ajustado con .values.reshape(-1, 1), así que las predicciones también necesitan esa forma
    # y luego se desescalan.
    # Si y_test es (num_samples, 1), entonces test_preds_scaled_np y y_test (labels) también lo son.
    test_preds_descaled = scaler.inverse_transform(test_preds_scaled_np)
    test_labels_descaled = scaler.inverse_transform(y_test) # y_test fue creado de test_data_scaled

    # Calcular RMSE en la escala original
    rmse_original_scale = math.sqrt(mean_squared_error(test_labels_descaled, test_preds_descaled))
    print(f"RMSE de previsión en escala original: {rmse_original_scale:.2f} pasajeros")

    # Graficar las previsiones vs los valores reales
    plt.figure(figsize=(15, 7))
    plt.plot(test_labels_descaled, label='Valores Reales (Prueba)', color='blue', marker='.')
    plt.plot(test_preds_descaled, label='Predicciones del Modelo (Prueba)', color='red', linestyle='--', marker='x')
    plt.title(f'Previsión de Pasajeros: {nombre_mejor_modelo_rnn_keras}', fontsize=16)
    plt.xlabel(f'Índice de Tiempo (en conjunto de prueba, desde {train_size - SEQUENCE_LENGTH + (TARGET_STEPS-1) })', fontsize=12)
    plt.ylabel(f'Número de {passengers_col_name}', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

    # Graficar sobre la serie temporal original (parte de prueba)
    # Necesitamos los índices de fecha correctos para la parte de prueba
    # El primer valor de y_test corresponde al índice (train_size + SEQUENCE_LENGTH) de la serie original
    start_idx_test_original = train_size + SEQUENCE_LENGTH + (TARGET_STEPS -1) # Ajuste para el índice inicial de la etiqueta
    
    # Asegurarse que la longitud de test_dates coincide con test_labels_descaled
    end_idx_test_original = start_idx_test_original + len(test_labels_descaled)
    test_dates = time_series_data.index[start_idx_test_original : end_idx_test_original]


    plt.figure(figsize=(15,7))
    plt.plot(time_series_data.index, time_series_data.values, label="Serie Original Completa", alpha=0.5)
    if len(test_dates) == len(test_labels_descaled):
        plt.plot(test_dates, test_labels_descaled, label='Valores Reales (Prueba)', color='blue', marker='.')
        plt.plot(test_dates, test_preds_descaled, label='Predicciones del Modelo (Prueba)', color='red', linestyle='--')
    else:
        print("Advertencia: Discrepancia de longitud entre test_dates y predicciones/etiquetas desescaladas. No se graficará la superposición detallada.")
        # Graficar solo las predicciones con índices numéricos si las fechas no coinciden
        plt.plot(range(len(test_labels_descaled)), test_labels_descaled, label='Valores Reales (Prueba) - Índice numérico', color='blue', marker='.')
        plt.plot(range(len(test_preds_descaled)), test_preds_descaled, label='Predicciones (Prueba) - Índice numérico', color='red', linestyle='--')


    plt.title(f'Previsión Superpuesta en Serie Original: {nombre_mejor_modelo_rnn_keras}', fontsize=16)
    plt.xlabel('Fecha', fontsize=12)
    plt.ylabel(f'Número de {passengers_col_name}', fontsize=12)
    plt.legend()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=12)) # Marcas cada 12 meses
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()
else:
    print("No se puede realizar inferencia: modelo no entrenado, datos de prueba no disponibles o scaler no definido.")

## **8. Guardar el Mejor Modelo (Keras)**

Guardaremos el modelo completo del mejor modelo RNN entrenado usando el formato nativo de Keras (`.keras`).

In [ ]:
# 8. Guardar el Mejor Modelo (Keras)
# ------------------------------------------------------------------------------

if mejor_modelo_rnn_keras is not None:
    modelos_rnn_guardados_dir_keras = "modelos_rnn_keras_guardados"
    if not os.path.exists(modelos_rnn_guardados_dir_keras):
        os.makedirs(modelos_rnn_guardados_dir_keras)

    timestamp_rnn_keras = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    nombre_archivo_limpio_rnn_keras = nombre_mejor_modelo_rnn_keras.replace(" (Keras)", "").replace(" ", "_")
    nombre_archivo_modelo_rnn_keras = f"{nombre_archivo_limpio_rnn_keras}_{timestamp_rnn_keras}_ts.keras" # Usar extensión .keras
    ruta_guardado_modelo_rnn_keras = os.path.join(modelos_rnn_guardados_dir_keras, nombre_archivo_modelo_rnn_keras)

    print(f"Guardando el modelo completo ({nombre_mejor_modelo_rnn_keras}) en: {ruta_guardado_modelo_rnn_keras}")
    mejor_modelo_rnn_keras.save(ruta_guardado_modelo_rnn_keras)
    print("Modelo RNN (Keras) guardado exitosamente.")

    # print("\nEjemplo de cómo cargar el modelo RNN (Keras) (comentado):")
    # from tensorflow.keras.models import load_model
    # modelo_cargado_rnn_keras = load_model(ruta_guardado_modelo_rnn_keras)
    # print(f"Modelo RNN '{nombre_mejor_modelo_rnn_keras}' cargado y listo para inferencia.")
    # modelo_cargado_rnn_keras.summary()
else:
    print("No hay un mejor modelo RNN para guardar (posiblemente por fallo en carga de datos o entrenamiento).")

## **Conclusiones y Próximos Pasos (RNN para Series Temporales en Keras)**

En este notebook, hemos construido un sistema de previsión de series temporales usando LSTMs y GRUs en Keras:
1.  Cargamos y exploramos el dataset "Air Passengers".
2.  Preprocesamos los datos: escalado y creación de secuencias de entrada-salida.
3.  Definimos modelos LSTM y GRU usando la API Sequential de Keras.
4.  Compilamos y entrenamos los modelos usando `model.fit()`.
5.  Realizamos previsiones y las comparamos con los valores reales, desescalando para la interpretación.
6.  Guardamos el modelo completo.

**Posibles Próximos Pasos:**
* **Multivariado:** Adaptar para series temporales multivariadas (múltiples features de entrada). `NUM_FEATURES` cambiaría.
* **Múltiples Pasos Futuros:** Modificar los modelos (`OUTPUT_SIZE_RNN`) y la creación de secuencias para predecir más de un paso adelante a la vez.
* **Atención:** Incorporar capas de atención (`tf.keras.layers.Attention` o `MultiHeadAttention`).
* **Modelos Transformer:** Explorar arquitecturas basadas en Transformers.
* **Hiperparámetros:** Ajustar `SEQUENCE_LENGTH`, `HIDDEN_SIZE_RNN`, número de capas, tasa de aprendizaje, optimizador, etc., usando herramientas como KerasTuner.
* **Callbacks:** Usar callbacks de Keras como `EarlyStopping` y `ModelCheckpoint` para mejorar el entrenamiento.
* **Análisis de Errores:** Investigar más a fondo los errores de predicción (ej. residuos).
* **Incorporar Features Exógenas:** Si hay variables adicionales que puedan influir en la serie temporal (ej. festivos, promociones).